In [242]:
import pandas as pd
import glob
import os
import numpy as np
from sklearn import linear_model

### 1. BE/ME Ratio in 2014
The size breakpoint for year t is the median NYSE market equity at the end of June of year t. BE/ME for June of year t is the book equity for the last fiscal year end in t-1 divided by ME for December of t-1. 

In [78]:
AShareEODDerivativeIndicator = pd.read_csv("/Users/yiliwang/Documents/ACR Internship/Supplementary datasets/EODDerivativeIndicator.csv", header=0)

#### Dataframe: Companies' ME on 20131231 (269 companies total)
dataframe name: ME_df_20131231

In [79]:
ME_20131231_list = AShareEODDerivativeIndicator[AShareEODDerivativeIndicator['TRADE_DT'] == 20131231]['S_VAL_MV'] # 269 stocks available at 20121231

In [80]:
# name list
ME_20131231_list = AShareEODDerivativeIndicator[AShareEODDerivativeIndicator['TRADE_DT'] == 20131231]['S_VAL_MV'] # 269 stocks available at 20121231

In [81]:
ME_df_20131231 = pd.DataFrame({'WINDCODE': S_INFO_WINDCODE_ME,'ME_20131231': ME_20131231_list})

In [82]:
#ME_df_20130630 = ME_df_20130630_0.set_index('WINDCODE') # set ticker as index
#ME_df_20130630.index.name = None

In [83]:
ME_df_20131231[:10] # preview the first 10 rows

,ME_20131231,WINDCODE
1248357,1.381897e+06,002465.SZ
1248358,2.920860e+06,601118.SH
1248359,1.252566e+07,601633.SH
1248360,1.544400e+05,600233.SH
1248361,4.049025e+06,600795.SH
1248362,1.230963e+07,601818.SH
1248363,3.294435e+06,603993.SH
1248364,1.700103e+06,600827.SH
1248365,1.509987e+06,600060.SH
1248366,6.471064e+05,600816.SH


#### Dataframe: Companies' BE on 20130930 (276 companies total)
dataframe name: BE_df_20130930

In [7]:
AShareBalanceSheet = pd.read_csv("/Users/yiliwang/Documents/ACR Internship/Supplementary datasets/BalanceSheet.csv", header=0)

In [8]:
BE_20130930_df  = AShareBalanceSheet[AShareBalanceSheet['REPORT_PERIOD']==20130930]
BE_20130930_T1 = BE_20130930_df[BE_20130930_df['STATEMENT_TYPE'] == 408001000] # 20120930 合并报表 BE, len = 274

In [9]:
WINDCODE_BE = BE_20130930_T1['S_INFO_WINDCODE']
BE_20130930_list = BE_20130930_T1['TOT_SHRHLDR_EQY_INCL_MIN_INT']

In [10]:
BE_df_20130930 = pd.DataFrame({'WINDCODE': WINDCODE_BE,'BE_20130930': BE_20130930_list})

In [11]:
# Merge dataframes of ME and BE together
BEME_ratio_df = BE_df_20130930.merge(ME_df_20131231, left_on='WINDCODE', right_on='WINDCODE', how='inner')

#### BEME_ratio_df has 268 companies.

In [12]:
# obtain BE/ME
BEME_ratio = BEME_ratio_df['BE_20130930']/BEME_ratio_df['ME_20131231']

In [13]:
# Add BE/ME ratio as a new column to dataframe BEME_ratio_df
se = pd.Series(BEME_ratio)
BEME_ratio_df["BE_ME_ratio"] = se.values

In [14]:
BEME_ratio_df = BEME_ratio_df[['WINDCODE', 'BE_20130930', 'ME_20131231', 'BE_ME_ratio']] # rearrange the order of columns

In [15]:
BEME_ratio_df[:10] # preview the first 10 rowsc

,WINDCODE,BE_20130930,ME_20131231,BE_ME_ratio
0,002475.SZ,2.801723e+09,1.829645e+06,1531.293618
1,002470.SZ,4.129302e+09,1.491000e+06,2769.484781
2,600036.SH,2.572740e+11,2.746441e+07,9367.540849
3,002146.SZ,1.128011e+10,1.890841e+06,5965.655755
4,601988.SH,9.256610e+11,7.313690e+07,12656.551909
5,000895.SZ,1.411834e+10,1.036032e+07,1362.731890
6,600048.SH,6.049754e+10,5.888845e+06,10273.243188
7,000718.SZ,4.495358e+09,9.337390e+05,4814.362528
8,002304.SZ,1.699384e+10,4.408560e+06,3854.736103
9,600031.SH,2.530321e+10,4.889796e+06,5174.696881


#### Normalize

In [16]:
def normalize_list(input_list):
    empty_list = [None]*len(input_list)
    for i in range(len(input_list)):
        empty_list[i] = (input_list[i]-input_list.mean())/input_list.std()
    return empty_list

In [17]:
norm_BEME_ratio = normalize_list(BEME_ratio_df['BE_ME_ratio'])

In [18]:
#for i in range(BEME_ratio_df.shape[0]):
#    norm_BEME_ratio[i] = (BEME_ratio_df['BE_ME_ratio'][i]-BEME_ratio_df['BE_ME_ratio'].mean())/BEME_ratio_df['BE_ME_ratio'].std()

In [19]:
# Add norm_BEME_ratio as a new column to dataframe BEME_ratio_df
se_norm = pd.Series(norm_BEME_ratio)
BEME_ratio_df["norm_BEME_ratio"] = se_norm.values

In [199]:
BEME_ratio_df[0:10]

,WINDCODE,BE_20130930,ME_20131231,BE_ME_ratio,norm_BEME_ratio
0,002475.SZ,2.801723e+09,1.829645e+06,1531.293618,-0.943715
1,002470.SZ,4.129302e+09,1.491000e+06,2769.484781,-0.663215
2,600036.SH,2.572740e+11,2.746441e+07,9367.540849,0.831511
3,002146.SZ,1.128011e+10,1.890841e+06,5965.655755,0.060847
4,601988.SH,9.256610e+11,7.313690e+07,12656.551909,1.576604
5,000895.SZ,1.411834e+10,1.036032e+07,1362.731890,-0.981901
6,600048.SH,6.049754e+10,5.888845e+06,10273.243188,1.036689
7,000718.SZ,4.495358e+09,9.337390e+05,4814.362528,-0.199968
8,002304.SZ,1.699384e+10,4.408560e+06,3854.736103,-0.417362
9,600031.SH,2.530321e+10,4.889796e+06,5174.696881,-0.118337


In [244]:
# Save df as csv for 2014 whole year use
BEME_ratio_df.to_csv("BEME_ratio_df_2014.csv")

### 2. Last Quarter Return

Duration: 20140331 - 20140630

return = [SP(20140630) - SP(20140331)]/SP(20140331)

sample pool: companies in BEME_ratio_df['WINDCODE']

#### a) Last Quarterly Return for Individual Company

In [21]:
##### Load in all the files of Stock Price

# set current working directory to the directory storing stock price data
os.chdir('/Users/yiliwang/Documents/ACR Internship/data/')

In [28]:
files = [file for file in os.listdir( os.curdir ) if file.endswith(".csv")]

In [31]:
# Store files to dictionary using their own name
frames = {}
for each_file in files:
    frames[os.path.splitext(each_file)[0]] = pd.read_csv(each_file)

In [32]:
# For instance, call '000002.SZ' stock data:
frames['000002.SZ'][:5]

,S_INFO_WINDCODE,TRADE_DT,S_DQ_OPEN,S_DQ_HIGH,S_DQ_LOW,S_DQ_CLOSE,S_DQ_VOLUME,S_DQ_ADJOPEN,S_DQ_ADJHIGH,S_DQ_ADJLOW,S_DQ_ADJCLOSE,S_DQ_ADJFACTOR,S_DQ_TRADESTATUS_ENG
0,000002.SZ,20130104,10.12,10.12,10.12,10.12,0.0,1163.93,1163.93,1163.93,1163.93,115.012731,SUSPEND
1,000002.SZ,20130107,10.12,10.12,10.12,10.12,0.0,1163.93,1163.93,1163.93,1163.93,115.012731,SUSPEND
2,000002.SZ,20130108,10.12,10.12,10.12,10.12,0.0,1163.93,1163.93,1163.93,1163.93,115.012731,SUSPEND
3,000002.SZ,20130109,10.12,10.12,10.12,10.12,0.0,1163.93,1163.93,1163.93,1163.93,115.012731,SUSPEND
4,000002.SZ,20130110,10.12,10.12,10.12,10.12,0.0,1163.93,1163.93,1163.93,1163.93,115.012731,SUSPEND


##### Extract Stock Price Data on Starting Day (20140331) and End Day (20140630)

In [33]:
# Obatin Close Price on 20140331 and 20140630
start_close = [None]*len(BEME_ratio_df['WINDCODE'])
end_close = [None]*len(BEME_ratio_df['WINDCODE'])

In [34]:
# Dataframe start_close: start_close price for 268 companies

for i in range(len(BEME_ratio_df['WINDCODE'])):
    company = BEME_ratio_df['WINDCODE'][i]
    start_close[i] =float(frames[company][frames[company]['TRADE_DT'] == 20140331]['S_DQ_ADJCLOSE'])
    end_close[i] = float(frames[company][frames[company]['TRADE_DT'] == 20140630]['S_DQ_ADJCLOSE'])

In [35]:
com_qtr_return = [None]*len(BEME_ratio_df['WINDCODE']) # create an empty list to store return for individual company

In [36]:
# Calculate quarterly return for 20140331 - 20140630
for i in range(len(BEME_ratio_df['WINDCODE'])):
    com_qtr_return[i] = (end_close[i] - start_close[i])/start_close[i]

In [248]:
# com_qtr_return

In [37]:
### Dataframe Summary: ###
# start_close:  close price on 20140331
# end_close: close price on 20140630
# BEME_ratio_df['WINDCODE']: company name
# com_qtr_return: quarterly return for the period 20140331- 20140630

# 268 observations for each df

#### b) Last Quarterly Return for CSI300 Index

In [38]:
csi300_IndexData= pd.read_csv("/Users/yiliwang/Documents/ACR Internship/csi_300_IndexData_daily_.csv", header=0)

In [39]:
index_start = float(csi300_IndexData[csi300_IndexData['date'] == '2014-03-31']['last_price']) # index price on start day: 2014-03-31
index_end = float(csi300_IndexData[csi300_IndexData['date'] == '2014-06-30']['last_price'])  # index price on end day: 2014-06-30

In [40]:
index_qtr_return = (index_end - index_start)/index_start # calculate the index quarterly return

In [41]:
print index_qtr_return

0.00649408125521


#### Difference = Company Return - Index Return 
Result List: diff_return_com_csi300

In [151]:
diff_return_com_csi300 = [None]*len(BEME_ratio_df['WINDCODE'])

In [152]:
for i in range(len(BEME_ratio_df['WINDCODE'])):
    diff_return_com_csi300[i] = com_qtr_return[i] - index_qtr_return

In [153]:
diff_return_com_csi300[0:10] # preview the head of the list

[0.272114614396962,
 0.007316660432767689,
 0.03609802744612069,
 -0.22416849985986306,
 0.05455243037269514,
 -0.06049438720595377,
 0.01095526878577652,
 0.04480049617517414,
 0.06460092679631796,
 -0.1000040580910715]

In [180]:
# combine all the lists of qtrly return into a large dataframe
Weighted_Return = pd.DataFrame({'Company':BEME_ratio_df['WINDCODE'],'start_price': start_close,'end_price': end_close,'qtr_return':com_qtr_return,'diff_csi300_LAST':diff_return_com_csi300})

In [181]:
# Normalize
norm_return_diff = normalize_list(Weighted_Return['diff_csi300_LAST'])

In [182]:
# Add a new column to dataframe BEME_ratio_df
se_norm_diff = pd.Series(norm_return_diff)
Weighted_Return["norm_return_diff_LAST"] = se_norm_diff.values

In [203]:
# Dataframe of Last Qtr Return of period 20140331-20140630
Weighted_Return[0:10] # preview

,Company,diff_csi300_LAST,end_price,qtr_return,start_price,norm_return_diff_LAST
0,002475.SZ,0.272115,147.04,0.278609,115.00,1.944860
1,002470.SZ,0.007317,19.82,0.013811,19.55,-0.229808
2,600036.SH,0.036098,39.90,0.042592,38.27,0.006560
3,002146.SZ,-0.224168,42.05,-0.217674,53.75,-2.130893
4,601988.SH,0.054552,3.65,0.061047,3.44,0.158118
5,000895.SZ,-0.060494,371.04,-0.054000,392.22,-0.786710
6,600048.SH,0.010955,86.88,0.017449,85.39,-0.199926
7,000718.SZ,0.044800,21.52,0.051295,20.47,0.078030
8,002304.SZ,0.064601,133.03,0.071095,124.20,0.240642
9,600031.SH,-0.100004,223.06,-0.093510,246.07,-1.111186


### 3. AShareEODDerivativeIndicator['FLOAT_A_SHR_TODAY']
total free-float shares per day * unadjusted close price

In [49]:
# Extract trading dates of certain period 
dates_trade = frames['000001.SZ']['TRADE_DT'].unique()
dates_trade.sort() # sort the date

In [50]:
start_dateind = list(dates_trade).index(20140331) # index of start date
end_dateind = list(dates_trade).index(20140630) # index of end date

In [51]:
# target_dates: list of trading dates in that period
target_dates = dates_trade[start_dateind:end_dateind+1]

In [52]:
target_dates

array([20140331, 20140401, 20140402, 20140403, 20140404, 20140408,
       20140409, 20140410, 20140411, 20140414, 20140415, 20140416,
       20140417, 20140418, 20140421, 20140422, 20140423, 20140424,
       20140425, 20140428, 20140429, 20140430, 20140505, 20140506,
       20140507, 20140508, 20140509, 20140512, 20140513, 20140514,
       20140515, 20140516, 20140519, 20140520, 20140521, 20140522,
       20140523, 20140526, 20140527, 20140528, 20140529, 20140530,
       20140603, 20140604, 20140605, 20140606, 20140609, 20140610,
       20140611, 20140612, 20140613, 20140616, 20140617, 20140618,
       20140619, 20140620, 20140623, 20140624, 20140625, 20140626,
       20140627, 20140630])

In [72]:
def find_FAST_mean(company): # For a given company, calculate mean of FLOAT_A_SHR_TODAY *close price (unadjusted) on trading days
    FAST_product = [None]*len(target_dates)
    
    for i in range(len(target_dates)):
        close_p = frames[company][frames[company]['TRADE_DT'] == target_dates[i]]['S_DQ_CLOSE'].values # get close price on a particular date
        
        # dataframe of the company
        df_company = AShareEODDerivativeIndicator[AShareEODDerivativeIndicator['S_INFO_WINDCODE'] ==company]
        FAST_day = float(df_company[df_company['TRADE_DT']==target_dates[i]]['FLOAT_A_SHR_TODAY']) #FLOAT_A_SHR_TODAY of a particular day
        
        FAST_product[i] = FAST_day*close_p
        
    FAST_mean = np.mean(FAST_product)
    return FAST_mean

In [73]:
find_FAST_mean('002475.SZ')

2246709.152032258

In [110]:
FAST_mean_list = [find_FAST_mean(company) for company in BEME_ratio_df['WINDCODE']]

In [111]:
FAST_mean_list # store 268 FAST means

[2246709.152032258,
 1138794.980322581,
 20782330.612576913,
 1789176.1934540325,
 52149533.012218677,
 4429710.2093945807,
 5402828.5449124845,
 628312.33325143554,
 4732042.4725486627,
 4054059.2441080646,
 4440218.2140538711,
 1454241.3817566938,
 578272.95251845161,
 7099207.7478426127,
 9915990.0283629056,
 1286461.3830989518,
 733026.71424193552,
 2863631.5664516124,
 2760891.9193548388,
 1290618.4505063226,
 3014246.0903225811,
 1880686.1838709677,
 454615.65179846768,
 2002545.3501224995,
 1873809.7865806455,
 412306.91079662903,
 5791029.881896209,
 1879789.6515700649,
 931317.80387464538,
 1890192.8177419356,
 1672865.1234375164,
 140090.75766856453,
 2746663.4261358702,
 878198.60288351635,
 8786958.2545161303,
 2906991.2261910005,
 1395012.9032258065,
 3499522.908239516,
 5432495.9432238089,
 2196327.8661840647,
 46682601.410197116,
 1749015.1950836452,
 5533167.6983899195,
 3895052.0232538553,
 3943909.6742990003,
 69991310.554312691,
 8842991.2409489676,
 353045.528518064

In [117]:
FAST_mean_df = pd.DataFrame({'Company':BEME_ratio_df['WINDCODE'],'FAST_mean':FAST_mean_list})

In [206]:
FAST_mean_df[0:10]

,Company,FAST_mean
0,002475.SZ,2.246709e+06
1,002470.SZ,1.138795e+06
2,600036.SH,2.078233e+07
3,002146.SZ,1.789176e+06
4,601988.SH,5.214953e+07
5,000895.SZ,4.429710e+06
6,600048.SH,5.402829e+06
7,000718.SZ,6.283123e+05
8,002304.SZ,4.732042e+06
9,600031.SH,4.054059e+06


In [249]:
FAST_mean_df.to_csv("FAST_mean_df_1")

## Y : Current Quarter Return

Duration: 20140630 - 20140930

return = [SP(20140930) - SP(20140630)]/SP(20140630)

sample pool: companies in BEME_ratio_df['WINDCODE']

#### a) Current Quarterly Return for Individual Company

##### Extract Stock Price Data on Starting Day (20140630) and End Day (20140930)

In [121]:
# Obatin Close Price on starting and end dates for current qtr
start_close_c = [None]*len(BEME_ratio_df['WINDCODE'])
end_close_c = [None]*len(BEME_ratio_df['WINDCODE'])

In [124]:
# Dataframe start_close: start_close price for 268 companies

for i in range(len(BEME_ratio_df['WINDCODE'])):
    company = BEME_ratio_df['WINDCODE'][i]
    start_close_c[i] =float(frames[company][frames[company]['TRADE_DT'] == 20140630]['S_DQ_ADJCLOSE'])
    end_close_c[i] = float(frames[company][frames[company]['TRADE_DT'] ==20140930]['S_DQ_ADJCLOSE'])

In [139]:
# create an empty list to store return for individual company for current qtr
com_qtr_return_CURRENT = [None]*len(BEME_ratio_df['WINDCODE']) 

In [140]:
# Calculate quarterly return for 20140331 - 20140630
for i in range(len(BEME_ratio_df['WINDCODE'])):
    com_qtr_return_CURRENT[i] = (end_close_c[i] - start_close_c[i])/start_close_c[i]

In [141]:
### Dataframe Summary: ### CURRENT QTR
# start_close_c:  close price on 20140630
# end_close_c: close price on 20140930
# BEME_ratio_df['WINDCODE']: company name
# com_qtr_return_CURRENT: quarterly return for the CURRENT period 

# 268 observations for each df
### CAN BE ENCAPSULATED INTO FUNCTION ###

#### b) CURRENT Quarterly Return for CSI300 Index

In [142]:
index_start_c = float(csi300_IndexData[csi300_IndexData['date'] == '2014-06-30']['last_price']) # index price on start day: 2014-03-31
index_end_c = float(csi300_IndexData[csi300_IndexData['date'] == '2014-09-30']['last_price'])  # index price on end day: 2014-06-30

In [143]:
index_qtr_return_CURRENT = (index_end_c - index_start_c)/index_start_c # calculate the index quarterly return

In [144]:
index_qtr_return_CURRENT

0.13150135847554495

#### Difference = Company Return - Index Return 
Result List: diff_return_com_csi300_CURRENT

In [193]:
diff_return_com_csi300_CURRENT = [None]*len(BEME_ratio_df['WINDCODE'])

In [194]:
for i in range(len(BEME_ratio_df['WINDCODE'])):
    diff_return_com_csi300_CURRENT[i] = com_qtr_return_CURRENT[i] - index_qtr_return_CURRENT

##### Y: diff_return_com_csi300_CURRENT

## Model Fit
Train set:

X1_matrix

Y1_matrix

In [235]:
Y = diff_return_com_csi300_CURRENT
X_1 = BEME_ratio_df['norm_BEME_ratio']
X_2 = Weighted_Return['norm_return_diff_LAST']
X_3 = FAST_mean_df['FAST_mean']

In [236]:
X1_matrix = np.column_stack((X_1, X_2, X_3)) # matrix X1: 268*3

In [237]:
Y1_matrix = np.array(Y)[:,np.newaxis] # matrix Y1: 268*1

In [238]:
clf = linear_model.LinearRegression()
clf.fit(X1_matrix,Y1_matrix)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [243]:
# Model generated
clf.coef_

array([[  2.23332311e-02,  -9.58030630e-03,  -3.72901600e-09]])

##### Testing Set: 
X1_test_arr

Y1_test_arr

In [267]:
X1_test = pd.read_csv("/Users/yiliwang/Documents/ACR Internship/Intermediate Files/Test_Set/X1_test.csv", header=0)
Y1_test = pd.read_csv("/Users/yiliwang/Documents/ACR Internship/Intermediate Files/Test_Set/Y1_test.csv", header=0)

In [275]:
# convert dataframes into arrays/matrix
X1_test_arr = np.column_stack((list(X1_test['0']), list(X1_test['1']), list(X1_test['2'])))
Y1_test_arr = np.array(list(Y1_test['0']))[:,np.newaxis] 

In [278]:
Y1_test_arr[0:10]

array([[-0.60020583],
       [-0.3644728 ],
       [ 0.15148358],
       [ 0.04633764],
       [ 0.08947707],
       [-0.50527673],
       [ 0.50418702],
       [-0.20842552],
       [-0.19323116],
       [ 0.26057807]])

In [285]:
# R^2
clf.score(X1_test_arr, Y1_test_arr, sample_weight=None)

-0.12276212462700831